Agora, aplicaremos todos os conceitos que vimos de otimização. Usaremos ParemeterGrid, ParameterSampler, GridSearch e RandomSearch. Use o datase diabetes.csv

# Instruções

In [1]:
from sklearn.metrics import log_loss, accuracy_score
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from scipy.stats import randint
from sklearn.model_selection import StratifiedKFold, ParameterGrid, ParameterSampler, GridSearchCV, RandomizedSearchCV, train_test_split

In [2]:
# dados 
df_diabetes = pd.read_csv('diabetes.csv')

# Treino, Teste e Validação

Primeiramente, teste a estratégia de dividir em treino, teste e validação. Ou seja, teremos 3 datasets. Siga o passo a passo abaixo.

1. Defina um Pipeline com Knn e MinMaxScaler. Defina também a validação StratifiedKFold
2. Divida o X em X_train, X_test e X_val. Sugestão de percentuais: 70, 15, 15
3. Crie a grid de parâmetros para o grid search (varie métrica de distância, número de vizinhos e pesos de votação)
4. Crie a grid de parâmetros para o random search (varie métrica de distância, número de vizinhos e pesos de votação). Use 50 iterações
5. Execute um loop para o grid search
6. Execute um loop para o random search
7. Avalie a performance e os conjuntos de hiperparametros associados (lembre-se de que o objetivo é pegar o conjunto, cujo erro de validação é o menor)

Dica: Divida primeiro em X_train e X_test e depois use o X_train para dividr em X_train e X_Val.
Lembre de printar os parâmetros escolhidos e o score. Use sempre 'neg_log_loss' como métrica.m

In [3]:
# Pipeline 
knn = Pipeline(steps=[('pre_processing', MinMaxScaler()), ('model', KNeighborsClassifier())])

In [4]:
# Divisão em X_train, X_test, X_val
df_train, df_test = train_test_split(df_diabetes, stratify=df_diabetes['class'], test_size=0.15, random_state=123)

In [5]:
n_rows_validation = int(0.15*df_diabetes.shape[0])
percent_validation = round((n_rows_validation/df_train.shape[0]), 2)

In [6]:
df_train, df_validation = train_test_split(df_train, stratify=df_train['class'], test_size=percent_validation, random_state=123)

In [7]:
# Definição da grid para o grid search
param_grid_search = {'model__metric':['manhattan','euclidean'],
                     'model__n_neighbors':list(range(1,52,2)),
                     'model__weights':['uniform','distance']}

In [8]:
# Definição da grid para o random search
param_random_search = {'model__metric':['manhattan', 'euclidean'],
                       'model__n_neighbors': randint(1,51),
                       'model__weights':['uniform','distance']}

In [9]:
X_train, y_train = df_train.drop('class', axis=1), df_train['class']
X_test, y_test = df_test.drop('class', axis=1), df_test['class']
X_validation, y_validation = df_validation.drop('class', axis=1), df_validation['class']

In [10]:
# Loop grid search
validation_score_grid_search = []
train_score_grid_search = []
list_grid_search_params = list(ParameterGrid(param_grid_search))
for combinacao in list_grid_search_params:
    knn.set_params(**combinacao)
    knn.fit(X_train, y_train)
    y_validation_predict = knn.predict_proba(X_validation)
    y_train_predict = knn.predict_proba(X_train)
    validation_score_grid_search.append(log_loss(y_validation, y_validation_predict))
    train_score_grid_search.append(log_loss(y_train, y_train_predict))

In [11]:
np.min(validation_score_grid_search)

0.48424436121745135

In [12]:
np.argmin(validation_score_grid_search)

79

In [13]:
best_grid_search_params = list_grid_search_params[np.argmin(validation_score_grid_search)]

In [14]:
knn_grid_search = knn.set_params(**best_grid_search_params)
knn_grid_search.fit(X_train, y_train)


Pipeline(steps=[('pre_processing', MinMaxScaler()),
                ('model',
                 KNeighborsClassifier(metric='euclidean', n_neighbors=27,
                                      weights='distance'))])

In [15]:
print(f'Performance no treino: {log_loss(y_train, knn_grid_search.predict_proba(X_train))}')

Performance no treino: 2.2204460492503136e-16


In [16]:
print(f'Performance no test: {log_loss(y_test, knn_grid_search.predict_proba(X_test))}')

Performance no test: 0.4617165016364378


In [17]:
df_metrics_grid_search = pd.DataFrame({'params':list_grid_search_params, 'train_score':train_score_grid_search, 'validation_score':validation_score_grid_search})

In [18]:
df_metrics_grid_search.sort_values(by='validation_score', inplace=True)

In [19]:
df_metrics_grid_search['validation_score']

79     0.484244
77     0.484616
76     0.484912
78     0.485690
75     0.487739
        ...    
3      4.884952
53    10.996369
52    10.996369
1     11.912733
0     11.912733
Name: validation_score, Length: 104, dtype: float64

In [20]:
len(list_grid_search_params)

104

In [21]:
# Loop Random Search
validation_score_random_search = []
train_score_random_search = []
list_random_search_params = list(ParameterSampler(param_random_search, n_iter=50, random_state=123))
for combinacao in list_random_search_params:
    knn.set_params(**combinacao)
    knn.fit(X_train, y_train)
    y_validation_predict = knn.predict_proba(X_validation)
    y_train_predict = knn.predict_proba(X_train)
    validation_score_random_search.append(log_loss(y_validation, y_validation_predict))
    train_score_random_search.append(log_loss(y_train, y_train_predict))

In [22]:
# Avalie os melhores parametros
np.min(validation_score_random_search)

0.48330469099035095

In [23]:
np.min(validation_score_grid_search)

0.48424436121745135

In [24]:
best_random_search_params = list_random_search_params[np.argmin(validation_score_random_search)]
knn_random_search = knn.set_params(**best_random_search_params)
knn_random_search.fit(X_train, y_train)
print(f'Performance no treino: {log_loss(y_train, knn_random_search.predict_proba(X_train))}')
print(f'Performance no test: {log_loss(y_test, knn_random_search.predict_proba(X_test))}')

Performance no treino: 0.4797081034836446
Performance no test: 0.4717777420818704


In [25]:
best_random_search_params

{'model__metric': 'euclidean',
 'model__n_neighbors': 24,
 'model__weights': 'uniform'}

In [26]:
knn_grid_search

Pipeline(steps=[('pre_processing', MinMaxScaler()),
                ('model',
                 KNeighborsClassifier(metric='euclidean', n_neighbors=24))])

# Cross Validation

Reaproveitando os passos 1, 2, 3, 4 do tópico 2, realize as validações cruzadas com Grid Search e Random Search. Lembre de printar os melhores parâmetros e o score obtido.

In [27]:
# Grid Search

In [28]:
# Random Search

In [29]:
# Avalie a performance no treino e na validação

OBS: Não estamos preocupado com performance. A ideia é exercitar a construção desses steps. O dataset não é tão grande e podemos sofrer com isso.